In [4]:
! pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


In [5]:
!git clone https://github.com/google-research/text-to-text-transfer-transformer.git

Cloning into 'text-to-text-transfer-transformer'...
remote: Enumerating objects: 4038, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 4038 (delta 73), reused 97 (delta 61), pack-reused 3897
Receiving objects: 100% (4038/4038), 5.47 MiB | 4.95 MiB/s, done.
Resolving deltas: 100% (2964/2964), done.


In [6]:
!pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 7.3 MB/s eta 0:00:00


In [12]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [13]:
books = books["train"].train_test_split(test_size=0.2)

In [14]:
books["train"][0]

{'id': '7903',
 'translation': {'en': 'I must not, however, neglect the duties of my station, or refrain from declaring my amazement at hearing that you received the young couple into your house as soon as they were married. It was an encouragement of vice; and had I been the rector of Longbourn, I should very strenuously have opposed it.',
  'fr': 'Je ne puis me dérober au devoir de ma charge et m’abstenir d’exprimer mon étonnement que vous ayez reçu le jeune couple chez vous, aussitôt apres le mariage : c’est un encouragement au vice, et si j’étais le recteur de Longbourn, je m’y serais opposé de tout mon pouvoir.'}}

In [15]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [16]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "

def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [17]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [18]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

##**Evaluate**

In [19]:
import evaluate

metric = evaluate.load("sacrebleu")

In [20]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

##**Train**

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_books["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [ ]:
model.save("/content/model")

##**Inference**

In [50]:
text = input("translate English to German: ")

print("Vstup:", text)


translate English to German: That is good.
Vstup: That is good.


In [51]:
from transformers import pipeline

translator = pipeline("translation", model="t5-base")

input_text = f"translate English to German: {text}"

result = translator(input_text)

output = {'inputs': input_text, 'targets': result[0]['translation_text']}

print(output)

{'inputs': 'translate English to German: That is good.', 'targets': 'Das ist gut so.'}
